# Intro

Herein we explore the generalized method of moments (GMM) using automatic differentiation (AD).  GMM is a likelihood-free way of estimating population parameters.  AD is kind of like symbolic differentiation; it is software that can create the gradient of an expression and evaluate it. 

We will be using the moment generating function (MGF) and cumulant generating function below, so it is useful to briefly review those now.  The MGF is defined as

$$
M(t) = \mathbb{E}[e^{t X}]
$$

and has the property $\mathbb{E}[X^k] = M^{(k)}(0)$ under certain regularity conditions.  The CGF is $K(t) = \log M(t)$, which has the nice property that $K'(0)$ is the mean and $K''(0)$ is the variance, i.e. the centered second moment, amongst other things.



# Automatic differentiation

We will use [Jax](https://jax.readthedocs.io/en/latest/index.html) for automatic differentiation (AD).  It is straightforward to use.  Below, we define the moment generating function and cumulant generating function of a normal distribution with mean and standard deviation $\theta = (\mu, \sigma)$.

We can then compute the gradient of these functions using `grad`.  The `grad` function will take the gradient of the first argument by default, but it is possible to specifiy which argument you want to differentiate.

We compute the first and second centered moment using the CGF and then evaluate those for a given $(\mu, \sigma)$ to confirm that this indeed is working as intended.



In [1]:
import jax.numpy as jnp
from jax import grad

In [2]:
def mgf_norm(t, theta):
    return jnp.exp(t * theta[0] + 0.5 * jnp.square(theta[1] * t))

def cgf_norm(t, theta):
    return t * theta[0] + 0.5 * jnp.square(theta[1] * t)

cgf1 = grad(cgf_norm)
cgf2 = grad(grad(cgf_norm))

In [3]:
theta0 = jnp.array([1.5, 2.0])
cgf1(0.0, theta0), cgf2(0.0, theta0)

(Array(1.5, dtype=float32, weak_type=True),
 Array(4., dtype=float32, weak_type=True))

# GMM for population parameter estimation

[Eric Zivot](https://faculty.washington.edu/ezivot) has an accessible [summary](https://faculty.washington.edu/ezivot/econ583/gmm.pdf) of GMM.  We recapitulate the essential parts here.

The idea is the following: you have $i = 1, \ldots, K$ functions, or moment conditions, that satisfy 

$$
\mathbb{E}[g_i(X; \theta)] = 0, i = 1, \ldots, K
$$

when $\theta = \theta_0$, the true value for the population.  (We will use capital letters like $X$ to denote a random variable and lower case letters like $x$ to denote a realization from the random variables distribution.)

The sample equivalent of this is

$$
\bar g_i(\theta) = \frac{1}{n} \sum_{t=1}^{n} g_i(x_t, \theta) = 0.
$$

Because we may have more moment conditions than parameters, we may not be able to solve this exactly using sampled data.  An obvious thing to do is to  minmize the squared error of these sample moment conditions.  Because it will be useful later, we actually want to think about minimizing the squared error using a symmetric, positive definite weighting matrix $W$.  That is, we want to minimize

$$
J_n(\theta, W) = n \bar g' W \bar g.
$$

The choice of $W$ can have a dramatic effect on the efficiency of the estimator, but, for any choice of $W$, solving for the $\theta$ that minimizes $J_n$ will produce a consistent estimate as $n \rightarrow \infty$.  The most efficient estimator is when $W$ is the inverse of the asymptotic variance of $\bar g$ as our data grows without bound, i.e. $n \rightarrow \infty$, under the true value $\theta = \theta_0$.  Assuming we have IID data,

$$
Var[ \frac{1}{n} \sum_{t=1}^{n} g(X_t, \theta) ] = \frac{1}{n} Var[ g(X_1, \theta) ].
$$

Let $\hat S$ be an estimator of the variance term,

$$
\hat S(\theta) = V_n(\theta) = \frac{1}{n} \sum_{i=1}^{n} g(x_i, \theta) g(x_i, \theta)'
$$

and let $\hat W = \hat S^{-1}$.  Then we can iteratively cylce through values of $\hat \theta$ and $\hat S$ by:

$$
\hat \theta = \underset{\theta}{\text{argmin}} \; J_n(\theta, \hat S)
$$

and

$$
\hat S = V_n(\hat \theta)
$$

to arrive at the estimator $\hat \theta$ that arises using the optimal weight matrix, approximately speaking.

Lastly, and critically, when we have $K$ moment conditions and only $L$ parameters, then asymptotically, $J_n$ converges to a $\chi^2_{K-L}$ distribution under the null hypothesis.  Thus, we can use $J_n$ as a statistic to create a p-value:

$$
p = 1 - CDF_{\chi^2}(J_n(\hat \theta, \hat S), \; df=K - L).
$$


# Moment conditions for estimating population parameters

For our purposes here, we adopt the moment conditions

$$
g_i(x, \theta) = x^i - M_\theta^{(i)}(0), i = 1, \ldots, 4.
$$

where $M^{(i)}$ is the $i$th derivative of the moment generating function with respect to $t$ and $\theta$ is the parameter vector.  AD makes computing $M^{(i)}$ trivial.

The code is very simple and can be found [here](https://github.com/jwindle/gmmfun).  We put that code to use below.  It is always useful to have an alternative approach that provides similar results as a way to check one's code, and to that end we also use the cumulant generating function to come up with moment conditions that can be used to find $\hat \theta$.


## Normal example

To give you an idea of how this works, let's estimate population parameters and the J-statistic when the null is true.  We use the classes `GmmMgf` and `GmmCgf` to estimate the population parameter using the GMM.  We have to provide the moment generating function (or cumulant generating function) and the bounds on the parameters, which in this case are the mean and standard deviation $\theta = (\mu, \sigma)$.

In [4]:
from gmmfun import GmmMgf4, GmmCgf4
from gmmfun.utils import sample_moments
from scipy.stats import norm, gamma, t, chi2, describe
import numpy as np

In [5]:
N = 100
np.random.seed(12345)
theta0 = jnp.array([2.0, 1.0])
x = norm.rvs(loc=theta0[0], scale=theta0[1], size=N)

In [6]:
np.mean(x), np.std(x)

(2.033614388208864, 1.0350846417161685)

In [7]:
lower_bounds = jnp.array([-jnp.inf, 0.0])
upper_bounds = jnp.array([ jnp.inf, jnp.inf])
bounds_norm = (lower_bounds, upper_bounds)

In [8]:
mgf_4 = GmmMgf4(mgf_norm, bounds_norm)
cgf_4 = GmmCgf4(cgf_norm, bounds_norm)
mgf_4.set_data(x)
cgf_4.set_data(x)

We initialize $\theta$ to the sample versions of the parameters and then initialize GMM using $I = W$.  You can see that the results are not identical, in terms of either $\theta$ or $J$, which is not surprising, since we, in effect, have two different $J$ functions at play.

In [9]:
theta_init = jnp.array([jnp.mean(x), jnp.std(x)])
mgf_4.opt_I(theta_init).params, cgf_4.opt_I(theta_init).params

(Array([2.0198934, 1.0553135], dtype=float32),
 Array([2.0428522, 1.051978 ], dtype=float32))

In [10]:
mgf_4.pval, cgf_4.pval

(0.9761345042539102, 0.9343752883331854)

We can then update $W$ as suggested above and then update $\theta$.  And following that, we update both multiple times.

In [11]:
mgf_4.update_W()

array([[ 47.258965  , -40.763363  ,  12.118055  ,  -1.150733  ],
       [-40.763355  ,  43.60277   , -14.460949  ,   1.4693025 ],
       [ 12.118056  , -14.460948  ,   5.068251  ,  -0.5336622 ],
       [ -1.1507338 ,   1.4693025 ,  -0.5336622 ,   0.05765441]],
      dtype=float32)

In [12]:
cgf_4.update_W()

array([[ 2.7704406 , -0.60642123, -0.5895131 ,  0.13728549],
       [-0.6064213 ,  2.6526172 ,  0.31469318, -0.35729185],
       [-0.58951306,  0.31469318,  0.19634886, -0.06248762],
       [ 0.13728549, -0.35729185, -0.06248762,  0.05759786]],
      dtype=float32)

In [13]:
foo = mgf_4.update_theta() # this returns the output of the optimization
foo = cgf_4.update_theta()

In [14]:
trace_mgf_4 = mgf_4.update_both(5)
trace_cgf_4 = cgf_4.update_both(5)

After updating both several times, we see that $\hat \theta$ and the $J$-stat are close.

In [15]:
mgf_4.theta, cgf_4.theta

(Array([2.0200768, 1.0357895], dtype=float32),
 Array([2.0214884, 1.035739 ], dtype=float32))

In [16]:
mgf_4.pval, cgf_4.pval

(0.8933808504104637, 0.8933015652026227)

## Gamma Example

To convince ourselves, that this was not a fluke, let us repeat the exercise using a gamma distribution.

In [17]:
def mgf_gamma(t, theta):
    return (1 - theta[1] * t)**(-theta[0])

def cgf_gamma(t, theta):
    return -theta[0] * jnp.log(1 - theta[1] * t)

In [18]:
N = 200
# np.random.seed(12348)
theta0 = jnp.array([5.0, 2.0])
x = gamma.rvs(theta0[0], scale=theta0[1], size=N)

In [19]:
# Shape, scale parameterization
lower_bounds = jnp.array([0.0, 0.0])
upper_bounds = jnp.array([ jnp.inf, jnp.inf])
bounds_gamma = (lower_bounds, upper_bounds)

In [20]:
mgf_4 = GmmMgf4(mgf_gamma, bounds_gamma)
cgf_4 = GmmCgf4(cgf_gamma, bounds_gamma)
mgf_4.set_data(x)
cgf_4.set_data(x)

In [21]:
mu_gamma, sig2_gamma = jnp.mean(x), jnp.var(x)
theta_init = jnp.array([mu_gamma**2 / sig2_gamma, sig2_gamma / mu_gamma])
mgf_4.opt_I(theta_init).params, cgf_4.opt_I(theta_init).params

(Array([2.9702945, 3.2302544], dtype=float32),
 Array([3.1960227, 3.1341908], dtype=float32))

In [22]:
mgf_4.pval, cgf_4.pval

(0.0, 0.0)

The one issue that arises here is that we can encounter issues of poor conditioning.  This is not surprising, since the size of the moments we are working with can vary greatly.  The old rule of thumb is that a condition number below 1e-15 is a very bad sign for double precision, but we aren't quite to that point yet.

In [23]:
trace_cgf_4 = cgf_4.update_both(5)

/Users/jwindle/Repos/gmmfun/src/gmmfun/gmm_base.py:110: LinAlgWarning: Ill-conditioned matrix (rcond=2.20426e-09): result may not be accurate.
  self.W = sp.linalg.solve(S, self.I, assume_a = "pos")
/Users/jwindle/Repos/gmmfun/src/gmmfun/gmm_base.py:110: LinAlgWarning: Ill-conditioned matrix (rcond=2.19965e-09): result may not be accurate.
  self.W = sp.linalg.solve(S, self.I, assume_a = "pos")
/Users/jwindle/Repos/gmmfun/src/gmmfun/gmm_base.py:110: LinAlgWarning: Ill-conditioned matrix (rcond=2.19909e-09): result may not be accurate.
  self.W = sp.linalg.solve(S, self.I, assume_a = "pos")


In [24]:
trace_mgf_4 = mgf_4.update_both(5)

/Users/jwindle/Repos/gmmfun/src/gmmfun/gmm_base.py:110: LinAlgWarning: Ill-conditioned matrix (rcond=1.7762e-12): result may not be accurate.
  self.W = sp.linalg.solve(S, self.I, assume_a = "pos")
/Users/jwindle/Repos/gmmfun/src/gmmfun/gmm_base.py:110: LinAlgWarning: Ill-conditioned matrix (rcond=1.54893e-12): result may not be accurate.
  self.W = sp.linalg.solve(S, self.I, assume_a = "pos")
/Users/jwindle/Repos/gmmfun/src/gmmfun/gmm_base.py:110: LinAlgWarning: Ill-conditioned matrix (rcond=1.54814e-12): result may not be accurate.
  self.W = sp.linalg.solve(S, self.I, assume_a = "pos")


You can see by just looking at the diagonal that the estimated variances are quite different in scale.

In [25]:
np.diag(cgf_4.W)

array([1.8355595e-01, 3.4637293e-03, 9.2738504e-05, 7.9921627e-08],
      dtype=float32)

In [26]:
np.diag(mgf_4.W)

array([2.0749578e+01, 2.4691960e-01, 4.3659454e-04, 7.9902030e-08],
      dtype=float32)

But ultimately it seems that the computations are ok since we end up with similar values for both approaches.

In [27]:
cgf_4.theta, mgf_4.theta

(Array([4.714462, 2.132177], dtype=float32),
 Array([4.7109103, 2.1361232], dtype=float32))

In [28]:
cgf_4.pval, mgf_4.pval

(0.4682780481912052, 0.46859687863293076)

## Scaling the moments for better numerical stability

One option for improving the numerical stability is to change the moment functions slightly, by re-scaling the higher moments.  When we do that, at least in our numerical experiments, we alleviate the issue.

In [29]:
from gmmfun import GmmMgf

discounts = np.array([10**k for k in range(1, 5)])
mgf_arb = GmmMgf(mgf_gamma, bounds_gamma, 4, weights=1.0/discounts)
mgf_arb.set_data(x)
mgf_arb.opt_I(theta_init).params

Array([3.199155 , 3.0612576], dtype=float32)

In [30]:
mgf_arb.update_both(5)
mgf_arb.theta, mgf_arb.pval

(Array([4.7079067, 2.136882 ], dtype=float32), 0.46879198620433815)

Here we can see that the variance components for the scaled moment conditions are more similar in scale.

In [31]:
np.diag(mgf_arb.W)

array([2071.2688   , 2464.7502   ,  435.82693  ,    7.9766316],
      dtype=float32)

## Arbitrary number of moments

Lastly, we have snuck in a new class here, `GmmMgf`.  This class works for an arbitrary number of moments, though as you increase the number of moments, you encounter the stability issues mentioned above.  The results are similar.

In [32]:
n_moments = 3
discounts = np.array([10**k for k in range(1, n_moments + 1)])
mgf_arb = GmmMgf(mgf_gamma, bounds_gamma, n_moments, weights=1.0/discounts)
mgf_arb.set_data(x)
mgf_arb.opt_I(theta_init).params, mgf_arb.pval

(Array([3.6667109, 2.7302873], dtype=float32), 0.8764779461175797)

In [33]:
trace_mgf_arb = mgf_arb.update_both(5)
[out.params for out in trace_mgf_arb]

[Array([4.645163, 2.149645], dtype=float32),
 Array([4.6497984, 2.147479 ], dtype=float32),
 Array([4.6497984, 2.147479 ], dtype=float32),
 Array([4.6497984, 2.147479 ], dtype=float32),
 Array([4.6497984, 2.147479 ], dtype=float32)]

In [34]:
mgf_arb.pval

0.3554125616864303

## Testing the alternative

Everything above was for testing when the null is true.  What happens when we use the gamma data, but fit it to a normal moment generating function?  We get a strong rejection.

In [35]:
discounts = np.array([10**k for k in range(1, 5)])
mgf_arb = GmmMgf(mgf_norm, bounds_norm, 4, weights=1.0/discounts)
mgf_arb.set_data(x)
mgf_arb.opt_I(theta_init).params

Array([8.980362 , 7.0330486], dtype=float32)

In [36]:
mgf_arb.update_both(5)
mgf_arb.theta, mgf_arb.pval

(Array([9.099801, 3.288657], dtype=float32), 3.0196674971394444e-05)

# Conclusion

The underlying code here is very minimal.  Using the power of automatic differentiation and the elegance of GMM, we were able to easily generate estimators of population parameters and measures of goodness of fit.  Be sure to check out the source code at <https://github.com/jwindle/gmmfun>.